## 자동차 번호판 검출하기
- 자동차 번호판 부분만 검출해 따로 저장한 후, 이미지 검색기로 원본 사진(자동차) 서칭

In [84]:
# 라이브러리 불러오기
import cv2
import matplotlib.pyplot as plt
import glob

import numpy as np
import time

In [85]:
import datetime
import PIL 
from PIL import Image

In [88]:
# 전체 이미지 불러올 경로 지정
img_paths = glob.glob('data/img/*.jpg') # glob 함수로 폴더 내 모든 jpg 파일을 불러옴
for img_path in img_paths: # img_paths에서 파일 경로 하나씩 꺼내오기
    img = cv2.imread(img_path) # 해당 경로에 있는 jpg 이미지 불러오기
    
    height, width, channel = img.shape # 뒤에서 쓸 높이, 너비, 채널 변수 설정
    
    # 그레이스케일 적용
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

    # Thresholding 하기
    blurred = cv2.GaussianBlur(gray, (3,3), 0) # 가우시안 블러링 
    img_th = cv2.adaptiveThreshold(blurred,
                              maxValue=255.0,
                              adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                              thresholdType=cv2.THRESH_BINARY_INV,
                              blockSize=19,
                              C=9) 

    # 해당 이미지의 모든 Contour 검출
    _, cnts, _=cv2.findContours(img_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    result = np.zeros((height, width, channel), dtype=np.uint8)
    
    # Contour 그려주기
    cv2.drawContours(result, contours=cnts, contourIdx=-1, color=(255, 0, 255))
    result = np.zeros((height, width, channel), dtype=np.uint8) 

    cnts_dict = []

    for cnt in cnts: # Contour 하나씩 꺼내기
        x, y, w, h = cv2.boundingRect(cnt) # 컨투어에 외접한 직사각형 좌표(x, y), 높이, 너비 리턴
        cv2.rectangle(result, pt1=(x, y), pt2=(x+w, y+h), color=(255, 0, 255), thickness=2)
        # 해당 좌표를 따라 사각형 그리기

        cnts_dict.append({
            'contour':cnt,
            'x':x,
            'y':y,
            'w':w,
            'h':h,
            'cx':x + (w/2),
            'cy':y + (h/2)}) # 그려진 사각형들의 컨투어와 좌표를 cnts_dict에 추가

    # 불필요한 contour 솎아내기
    MIN_AREA = 80
    MIN_WIDTH, MIN_HEIGHT = 2, 8
    MIN_RATIO, MAX_RATI0 = 0.25, 1.0 # 최소, 최댓값 지정

    possible_cnts = [] # 필요한 contour만 담을 그릇 만들기

    cnt = 0
    for d in cnts_dict:
        area = d['w']*d['h'] # contour 사각형 넓이
        ratio = d['w']/d['h'] # contour 사각형 너비/높이 비율

        if area > MIN_AREA \
        and d['w'] > MIN_WIDTH and d['h'] > MIN_HEIGHT \
        and MIN_RATIO < ratio < MAX_RATI0:
            d['idx'] = cnt
            cnt += 1
            possible_cnts.append(d) # 조건에 맞는 Contour만 저장해주기

    result = np.zeros((height, width, channel), dtype=np.uint8)

    for d in possible_cnts:
        cv2.rectangle(result, 
                      pt1=(d['x'], d['y']), 
                      pt2=(d['x']+d['w'], d['y']+d['h']),
                      color=(255, 0 , 255),
                      thickness=2) # 조건에 맞는 Contour 사각형만 그려 표시하기
    
    # 차량 번호판 contour만 검출하기 위한 기준값들 지정
    MAX_DIAG_MULTIPLYER = 5
    MAX_ANGLE_DIFF = 12.0
    MAX_AREA_DIFF = 0.5
    MAX_WIDTH_DIFF = 0.8
    MAX_HEIGHT_DIFF = 0.2
    MIN_N_MATCHED = 3

    def find_chars(cnt_list):
        matched_result_idx = [] # 최종 매칭 Contour들을 담을 그릇

        for d1 in cnt_list:
            matched_cnts_idx = [] # 매칭되는 Contour들의 index를 담을 그릇
            for d2 in cnt_list:
                if d1['idx'] == d2['idx']:
                    continue

                dx = abs(d1['cx'] - d2['cx'])
                dy = abs(d1['cy'] - d2['cy']) # 각 Contour들의 거리 구하기

                diagonal_length1 = np.sqrt(d1['w']**2 + d1['h']**2) # Contour 사각형 대각선 길이 구하기, 피타고라스 정리 활용 
                distance = np.linalg.norm(np.array([d1['cx'], d1['cy']]) - np.array([d2['cx'], d2['cy']]))
                
                if dx == 0:
                    angle_diff = 90
                else:
                    angle_diff = np.degrees(np.arctan(dy/dx)) # 이미지 기울어졌을 경우 각도 계산해주기

                area_diff = abs(d1['w']*d1['h'] - d2['w']*d2['h']) / (d1['w']*d1['h'])
                width_diff = abs(d1['w'] - d2['w']) / d1['w']
                height_diff = abs(d1['h'] - d2['h']) / d1['h']

                if distance < diagonal_length1 * MAX_DIAG_MULTIPLYER \
                and angle_diff < MAX_ANGLE_DIFF and area_diff < MAX_AREA_DIFF \
                and width_diff < MAX_WIDTH_DIFF and height_diff < MAX_HEIGHT_DIFF:
                    matched_cnts_idx.append(d2['idx']) # 조건에 맞는 Contour들의 idx만 추가해주기 

            matched_cnts_idx.append(d1['idx'])

            if len(matched_cnts_idx) < MIN_N_MATCHED:
                continue

            matched_result_idx.append(matched_cnts_idx)

            unmatched_cnt_idx = [] # 매칭되지 않는 Contour 그릇 만들기
            for d4 in cnt_list:
                if d4['idx'] not in matched_cnts_idx:
                    unmatched_cnt_idx.append(d4['idx'])

            unmatched_cnt = np.take(possible_cnts, unmatched_cnt_idx)

            recursive_cnt_list = find_chars(unmatched_cnt)

            for idx in recursive_cnt_list:
                matched_result_idx.append(idx)

            break

        return matched_result_idx # 최종 매칭 Contour Index 돌려주기

    result_idx = find_chars(possible_cnts)

    matched_result = []
    for idx_list in result_idx:
        matched_result.append(np.take(possible_cnts, idx_list))

    result = np.zeros((height, width, channel), dtype=np.uint8)

    for r in matched_result:
        for d in r:
            cv2.rectangle(result, 
                          pt1=(d['x'], d['y']), 
                          pt2=(d['x']+d['w'], d['y']+d['h']), 
                          color=(255, 0, 255),
                         thickness=2) # 최종 매칭된 Contour들에 사각형 그리기

    # 기준값 지정하기
    PLATE_WIDTH_PADDING = 1.3
    PLATE_HEIGHT_PADDING = 1.5
    MIN_PLATE_RATIO = 3
    MAX_PLATE_RATIO = 10

    plate_imgs = []
    plate_infos = []

    for i, matched_chars in enumerate(matched_result):
        sorted_chars = sorted(matched_chars, key=lambda x:x['cx']) 

        plate_cx = (sorted_chars[0]['cx'] + sorted_chars[-1]['cx']) / 2 # 중간점 x좌표 지정
        plate_cy = (sorted_chars[0]['cy'] + sorted_chars[-1]['cy']) / 2 # 중간점 y좌표 지정

        plate_width = (sorted_chars[-1]['x'] + sorted_chars[-1]['w'] - sorted_chars[0]['x'])*PLATE_WIDTH_PADDING 
        # 검출할 부분의 너비 구하기

        sum_height = 0
        for d in sorted_chars:
            sum_height += d['h']

        plate_height = int(sum_height / len(sorted_chars)*PLATE_HEIGHT_PADDING) # 검출할 지점 높이 구하기

        triangle_height = sorted_chars[-1]['cy'] - sorted_chars[0]['cy']
        triangle_hypotenus = np.linalg.norm(np.array([sorted_chars[0]['cx'], sorted_chars[0]['cy']]) - 
                                           np.array([sorted_chars[-1]['cx'], sorted_chars[-1]['cy']]))

        angle = np.degrees(np.arcsin(triangle_height / triangle_hypotenus)) # 이동할 각도 구하기

        rotation_mtrx = cv2.getRotationMatrix2D(center=(plate_cx, plate_cy), angle=angle, scale=1.0) 
        img_rotated = cv2.warpAffine(img_th, M=rotation_mtrx, dsize=(width, height)) # 원근 변환 
        
        # 번호판 이미지 검출
        img_cropped = cv2.getRectSubPix(img_rotated,
                                       patchSize = (int(plate_width), int(plate_height)), 
                                       center=(int(plate_cx), int(plate_cy))) # img_rotated 대신 img를 넣을 경우 컬러로 검출됨

        if img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO \
        or img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO > MAX_PLATE_RATIO:
            continue

        plate_imgs.append(img_cropped)
        plate_infos.append({
            'x':int(plate_cx - plate_width / 2),
            'y':int(plate_cy - plate_height / 2),
            'w':int(plate_width),
            'h':int(plate_height)})
        
        now = datetime.datetime.now()
        now_str = now.strftime("%H.%M.%S.%f")
        finished = Image.fromarray(img_cropped) # numpy array 자료형을 image 객체로 변환해준다
        finished.save("data/n/{}.jpg".format(now_str)) # 지정한 폴더에 저장하기
        # 컬러 이미지는 data/num2에 따로 저장      

- 전처리한 이미지
***
![result](https://user-images.githubusercontent.com/58945760/72497450-e2740100-386f-11ea-8482-18bae7104d13.png)


- 컬러 이미지
***
![num2](https://user-images.githubusercontent.com/58945760/72580186-959b3380-391e-11ea-99ba-d11f4d77ea92.png)




## 이미지 검색기 만들기(1차)

- Searching이 진행되긴 하나 추출기가 번호판과 차량 이미지 사이에 매칭점을 찾지 못해 error가 발생한다. 

In [71]:
import glob, cv2, numpy as np

In [81]:
# 검출한 번호판 이미지로 검색기 돌려보기
detector = cv2.ORB_create()
FLANN_INDEX_LSH = 6
index_params = {'algorithm' : FLANN_INDEX_LSH, 'table_number' : 6, 'key_size' : 12, 'multi_probe_level' : 1}
search_params = {'checks': 32}
matcher = cv2.FlannBasedMatcher(index_params, search_params)

def serch(img):
    kp1, desc1 = detector.detectAndCompute(img, None)

    results = {}

    img_paths = glob.glob('data/img/*.jpg')
    for img_path in img_paths:
        cars = cv2.imread(img_path)
        cv2.imshow('searching..', cars)
        cv2.waitKey(10)

        gray2 = cv2.cvtColor(cars, cv2.COLOR_BGR2GRAY)
        kp2, desc2 = detector.detectAndCompute(gray2, None)
        matches = matcher.knnMatch(desc1, desc2, 2)
        
        ratio = 0.7
        good_matches = [m[0] for m in matches if len(m) == 2 and m[0].distance < m[1].distance*ratio] 
        
        MIN_MATCH = 10
        if len(good_matches) > MIN_MATCH:
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches])
            mtrx, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

            accuracy = float(mask.sum()) / mask.size

            results[img_path] = accuracy
 #       cv2.destroyAllWindows('serching..')

    if len(results) > 0 :
        results = sorted([(v, k) for (k, v) in results.items() if v>0], reverse=True)

        return results

img_test = cv2.imread('data/num2/17.57.40.996459.jpg') # 번호판 이미지를 넣을 때 매칭이 안되는 오류 발생
gray = cv2.cvtColor(img_test, cv2.COLOR_BGR2GRAY)
results = serch(gray)


if type(results) is None :
    print("NO matched cars found")
else:
    for(i, (accuracy, img_path)) in enumerate(results):
        print(i, img_path, accuracy)
        if i == 0:
            cars = cv2.imread(img_path)
            cv2.putText(cars, ("Accuracy:%.2f%%"%(accuracy*100)), (10,100),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)
cv2.imshow('result', cars)
cv2.waitKey()
cv2.destroyAllWindows()

## 매칭점 확인으로 적합한 추출기 선정
- 강의에서 배웠던 추출기들을 활용하여 번호판 이미지와 매칭점을 찾아낼 수 있는 추출기를 찾는다.  

### 1. BF + ORB(실패)

In [32]:
import cv2, numpy as np

img1 = cv2.imread('data/img/car_plate10.jpg')
img2 = cv2.imread('data/num2/17.57.40.996459.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# ORB 서술자 추출기 생성 ---①
detector = cv2.ORB_create()
# 각 영상에 대해 키 포인트와 서술자 추출 ---②
kp1, desc1 = detector.detectAndCompute(gray1, None)
kp2, desc2 = detector.detectAndCompute(gray2, None)

# BFMatcher 생성, Hamming 거리, 상호 체크 ---③
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
print(matcher)

# 매칭 계산 ---④
matches = matcher.match(desc1, desc2)
# 매칭 결과 그리기 ---⑤
res = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, \
                     flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

cv2.imshow('BFMatcher + ORB', res)
cv2.waitKey()
cv2.destroyAllWindows()

<BFMatcher 000001E9D9DBEB70>


error: OpenCV(3.4.2) C:\projects\opencv-python\opencv\modules\core\src\batch_distance.cpp:238: error: (-215:Assertion failed) type == src2.type() && src1.cols == src2.cols && (type == 5 || type == 0) in function 'cv::batchDistance'


### 2. BF + SIFT (실패)

In [27]:
import cv2, numpy as np

img1 = cv2.imread('data/img/car_plate10.jpg')
img2 = cv2.imread('data/num2/17.57.40.996459.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

## SIFT 서술자 추출기 생성 ---①
detector = cv2.xfeatures2d.SIFT_create()
# 각 영상에 대해 키 포인트와 서술자 추출 ---②
kp1, desc1 = detector.detectAndCompute(gray1, None)
kp2, desc2 = detector.detectAndCompute(gray2, None)

# BFMatcher 생성, L1 거리, 상호 체크 ---③
matcher = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)
# 매칭 계산 ---④
matches = matcher.match(desc1, desc2)
# 매칭 결과 그리기 ---⑤
res = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, \
                      flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
# 결과 출력 
cv2.imshow('BFMatcher + SIFT', res)
cv2.waitKey()
cv2.destroyAllWindows()

None


error: OpenCV(3.4.2) c:\projects\opencv-python\opencv\modules\imgproc\src\color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper'


### 3. BF + SURF(성공)

In [94]:

import cv2
import numpy as np

img1 = cv2.imread('data/img/car_plate100.jpg')
img2 = cv2.imread('data/num/13.43.55.925497.jpg')
img3 = cv2.imread('data/num2/17.57.41.032328.jpg')

gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
gray3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)

# SURF 서술자 추출기 생성 ---①
detector = cv2.xfeatures2d.SURF_create()
kp1, desc1 = detector.detectAndCompute(gray1, None)
kp2, desc2 = detector.detectAndCompute(gray2, None)
kp3, desc3 = detector.detectAndCompute(gray3, None)

# BFMatcher 생성, L2 거리, 상호 체크 ---③
matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

# 매칭 계산 ---④
matches = matcher.match(desc1, desc2)
# 매칭 결과 그리기 ---⑤
res1 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, \
                     flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

cv2.imshow('BF + SURF1', res1) # 전처리한 이미지 매칭
cv2.waitKey()
cv2.destroyAllWindows()


# 매칭 계산 ---④
matches = matcher.match(desc1, desc3)
# 매칭 결과 그리기 ---⑤
res2 = cv2.drawMatches(img1, kp1, img3, kp3, matches, None, \
                     flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

cv2.imshow('BF + SURF2', res2) # 컬러 이미지 매칭
cv2.waitKey()
cv2.destroyAllWindows()


- 전처리한 이미지 매칭점
![match](https://user-images.githubusercontent.com/58945760/72691461-ef8f3980-3b68-11ea-905f-cd9aae407317.PNG)

- 컬러 이미지 매칭점
![match2](https://user-images.githubusercontent.com/58945760/72691488-2c5b3080-3b69-11ea-8476-3bd122834bd2.PNG)

### 4. FLANN + SURF(실패)

In [95]:
import cv2, numpy as np

img1 = cv2.imread('data/img/car_plate100.jpg')
img2 = cv2.imread('data/num2/17.57.41.032328.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# SURF 생성
detector = cv2.xfeatures2d.SURF_create()
# 키 포인트와 서술자 추출
kp1, desc1 = detector.detectAndCompute(gray1, None)
kp2, desc2 = detector.detectAndCompute(gray2, None)

# 인덱스 파라미터와 검색 파라미터 설정 ---①
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

# Flann 매처 생성 ---③
matcher = cv2.FlannBasedMatcher(index_params, search_params)
# 매칭 계산 ---④
matches = matcher.match(desc1, desc2)
# 매칭 그리기
res = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, \
                flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

cv2.imshow('Flann + SURF', res)
cv2.waitKey()
cv2.destroyAllWindows()

![match3](https://user-images.githubusercontent.com/58945760/72691526-704e3580-3b69-11ea-9d3c-8bc34878f5ae.PNG)


## 이미지 검색기 만들기(2차)
- 원래 코드에 쓰였던 ORB+KNN+FLANN 추출기를 SURF+BF 추출기로 변경

In [99]:
# 검출한 번호판 이미지로 검색기 돌려보기

detector = cv2.xfeatures2d.SURF_create()
matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

def serch(img):
#     gray1 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp1, desc1 = detector.detectAndCompute(gray1, None)

    results = {}

    img_paths = glob.glob('data/img/*.jpg')
    for img_path in img_paths:
        cars = cv2.imread(img_path)
        cv2.imshow('searching..', cars)
        cv2.waitKey(5)

        gray2 = cv2.cvtColor(cars, cv2.COLOR_BGR2GRAY)
        kp2, desc2 = detector.detectAndCompute(gray2, None)
        matches = matcher.match(desc1, desc2)

        # 매칭 결과를 거리기준 오름차순으로 정렬 ---③
        matches = sorted(matches, key=lambda x:x.distance)

        # 최소 거리 값과 최대 거리 값 확보 ---④
        min_dist, max_dist = matches[0].distance, matches[-1].distance
        # 최소 거리의 20% 지점을 임계점으로 설정 ---⑤
        ratio = 0.2
        good_thresh = (max_dist - min_dist) * ratio + min_dist

        # 임계점 보다 작은 매칭점만 좋은 매칭점으로 분류 ---⑥
        #m.distance 매칭객체의 거리 공통함수부분참조  
        good_matches = [m for m in matches if m.distance < good_thresh]
        
        MIN_MATCH = 10
        if len(good_matches) > MIN_MATCH:
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches])
            mtrx, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

            accuracy = float(mask.sum()) / mask.size

            results[img_path] = accuracy
 #       cv2.destroyAllWindows('serching..')

    if len(results) > 0 :
        results = sorted([(v, k) for (k, v) in results.items() if v > 0], reverse=True)

        return results

img_test = cv2.imread('data/num2/17.57.40.996459.jpg') 
gray = cv2.cvtColor(img_test, cv2.COLOR_BGR2GRAY)
results = serch(gray)


if type(results) is None :
    print("NO matched cars found")
else:
    for(i, (accuracy, img_path)) in enumerate(results):
#         print(i, img_path, accuracy)
        if i == 0:
            cars = cv2.imread(img_path)
            cv2.putText(cars, ("Accuracy:%.2f%%"%(accuracy*100)), (10,100),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('number', img_test)              
cv2.imshow('result', cars)
cv2.waitKey()
cv2.destroyAllWindows()

## 이미지 검색기 만들기(3차)
- Good matches를 뽑아내지 않고 전체 matches만으로 이미지 검출

In [101]:
# 검출한 번호판 이미지로 검색기 돌려보기((Good match 사용하지 않음)

detector = cv2.xfeatures2d.SURF_create()
matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

def serch(img):
    kp1, desc1 = detector.detectAndCompute(gray1, None)

    results = {}

    img_paths = glob.glob('data/img/*.jpg')
    for img_path in img_paths:
        cars = cv2.imread(img_path)
        cv2.imshow('searching..', cars)
        cv2.waitKey(3)

        gray2 = cv2.cvtColor(cars, cv2.COLOR_BGR2GRAY)
        kp2, desc2 = detector.detectAndCompute(gray2, None)
        matches = matcher.match(desc1, desc2)
        
        MIN_MATCH = 10
        if len(matches) > MIN_MATCH:
            src_pts = np.float32([kp1[m.queryIdx].pt for m in matches])
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches])
            mtrx, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

            accuracy = float(mask.sum()) / mask.size

            results[img_path] = accuracy
 #       cv2.destroyAllWindows('serching..')

    if len(results) > 0 :
        results = sorted([(v, k) for (k, v) in results.items() if v > 0], reverse=True)

        return results

img_test = cv2.imread('data/num2/17.57.41.032328.jpg') 
gray = cv2.cvtColor(img_test, cv2.COLOR_BGR2GRAY)
results = serch(gray)


if type(results) is None :
    print("NO matched cars found")
else:
    for(i, (accuracy, img_path)) in enumerate(results):
#         print(i, img_path, '%.2f'%accuracy)
        if i == 0:
            cars = cv2.imread(img_path)
            cv2.putText(cars, ("Accuracy:%.2f%%"%(accuracy*100)), (10,100),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)

cv2.imshow('number', img_test)            
cv2.imshow('result', cars)
cv2.waitKey()
cv2.destroyAllWindows()

![match4](https://user-images.githubusercontent.com/58945760/72691519-5f9dbf80-3b69-11ea-8b46-e659ca432108.PNG)


## (Extra)Tesseract로 글자 인식해보기

In [ ]:
import glob
import PIL
import pytesseract

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'

print()
img_list = []
number_list = []
error_list = []

img_paths = glob.glob('data/num/*.jpg') # glob 함수로 폴더 내 모든 jpg 파일을 불러옴
for img_path in img_paths: # img_paths에서 파일 경로 하나씩 꺼내오기
    img = cv2.imread(img_path)
    img_list.append(img)
    number = pytesseract.image_to_string(img, lang = 'eng')
    
    if number == '':
#         print('차량 번호 인식에 실패했습니다.')
        error_list.append(number)
    
    else:
#         print('차량 번호:', number)
        number_list.append(number)
        
    result = len(number_list) / len(img_list)
print()    
print('차량 번호 검출율:', '%.2f'%result)

## 참고 페이지

In [ ]:
# https://kolikim.tistory.com/44
# https://stackoverflow.com/questions/902761/saving-a-numpy-array-as-an-image/19174800
# https://github.com/kairess/license_plate_recognition/blob/master/main.ipynb